# Le modèle de l'Oregonator 

Le modèle de l'Oregonator s'écrit sous la forme :

\begin{equation}
\left\{
\begin{aligned}
{\mathrm d}_t y_1 & = y_2-y_1\\
\varepsilon\,  {\mathrm d}_t y_2 &= q\,y_3-y_3\,y_2+y_2(1-y_2)\\
\mu\,  {\mathrm d}_t y_3 &= -q\,y_3- y_3\,y_2+f\, y_1
\end{aligned}
\right.
\end{equation}

On prendra les paramètres suivants :

\begin{equation} 
\epsilon = 10^{-2},\quad \mu =10^{-4}, \quad f=1,\qquad q=2.10^{-4}.
\end{equation}

In [ ]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from mylib.model import oregonator_model
import mylib.integration as integration

import time

output_notebook(hide_banner=True)

## Solution quasi-exacte 

La solution quasi-exacte est obtenue en utilisant la méthode Radau d'ordre 5.

In [ ]:
def plot_quasi_exact_sol():
    
    eps = 1.e-2
    mu = 1.e-4
    f = 1.
    q = 2.e-4
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn = om.fcn
    jac = om.jac
    
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, jac=jac)
    
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.line(sol.t, sol.y[0], line_width=2)

    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.line(sol.t, sol.y[1], line_width=2)    

    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y3")
    fig_sol_y3.line(sol.t, sol.y[2], line_width=2)    

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))   
    
plot_quasi_exact_sol()

## Caractérisation de la raideur 

In [ ]:
def plot_eigenvalue():

    eps = 1.e-2
    mu = 1.e-4
    f = 1.
    q = 2.e-4
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn = om.fcn
    jac = om.jac
    
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, jac=jac)
    
    eig_vals = np.zeros((sol.t.size, 3), dtype=np.complex_)
    for it, yi in enumerate(sol.y.transpose()):
        eig_vals[it] = np.linalg.eigvals(jac(0, yi))

    lambda1 = eig_vals[:, 0]
    lambda2 = eig_vals[:, 1]
    lambda3 = eig_vals[:, 2]
    
    fig_real = figure(x_range=(tini, tend), plot_height=300, plot_width=950, 
                      title = "Real part of eigenvalues (click on legend entries to hide corresponding plot)")
    fig_imag = figure(x_range=(tini, tend), plot_height=300, plot_width=950, 
                      title = "Imaginary part of eigenvalues (click on lengend to hide corresponding plot)")
            
    fig_real.line(sol.t, np.real(lambda1), line_width=2, legend="lambda1")
    fig_real.line(sol.t, np.real(lambda2), line_width=2, color="Green", legend="lambda2")
    fig_real.line(sol.t, np.real(lambda3), line_width=2, color="Crimson", legend="lambda3")
    fig_real.legend.click_policy="hide"

    fig_imag.line(sol.t, np.imag(lambda1), line_width=2, legend="lambda1")
    fig_imag.line(sol.t, np.imag(lambda2), line_width=2, color="Green", legend="lambda2")
    fig_imag.line(sol.t, np.imag(lambda3), line_width=2, color="Crimson", legend="lambda3")
    fig_imag.legend.click_policy="hide"

    show(column(fig_real, fig_imag))
    
plot_eigenvalue()

## Méthode Radau5

In [ ]:
def plot_radau5_sol():
    
    eps = 1.e-2
    mu = 1.e-4
    f = 1.
    q = 2.e-4
    
    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn = om.fcn
    jac = om.jac
    
    tini = 0. 
    tend = 30.
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))
    
    tol = 1.e-6
    sol_rad = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=tol, atol=tol, jac=jac)
    
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(sol_rad.t, sol_rad.y[0], line_width=2)

    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(sol_rad.t, sol_rad.y[1], line_width=2)    

    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y3")
    fig_sol_y3.x(sol_rad.t, sol_rad.y[2], line_width=2)    

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))   
    
plot_radau5_sol()

## Runge-Kutta (order 4)

In [ ]:
def show_rk4_sol():

    eps = 1.e-2
    mu  = 1.e-5
    f   = 1.
    q   = 2.e-4

    om = oregonator_model(eps=eps, mu=mu, f=f, q=q)
    fcn = om.fcn

    tini = 0.
    tend = 10
    
    yini = (0.5, 1.e-3, (f*0.5)/(q+1.e-3))

    nt = 500001
    sol_rk4 = integration.rk4(tini, tend, nt, yini, fcn)
    
    
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(sol_rk4.t, sol_rk4.y[0], line_width=2)
 
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(sol_rk4.t, sol_rk4.y[1], line_width=2)
    
    fig_sol_y3 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y3")
    fig_sol_y3.x(sol_rk4.t, sol_rk4.y[2], line_width=2)

    show(column(fig_sol_y1, fig_sol_y2, fig_sol_y3))
    
show_rk4_sol()